In [0]:
#this helps build the configuration from the pyproject.toml file

In [0]:
%pip install -e ..

In [0]:
%pip install git+https://github.com/end-to-end-mlops-databricks-3/marvelous

In [0]:
%restart_python

In [0]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent / 'src'))

In [0]:
import pandas as pd
import yaml
from pyspark.sql import SparkSession
from loguru import logger
from hotel_reservations.config import ProjectConfig
from hotel_reservations import DataReader,DataProcessor
from marvelous.logging import setup_logging
from marvelous.timer import Timer

In [0]:
#ideally i would want to save the lgos in volumes, but that did not work, unless i save it into a temp dir first

In [0]:
config_path = f"../project_config.yml"

config = ProjectConfig.from_yaml(config_path=config_path,env="dev")

temp_log_file_path = "/tmp/logs/marvelous-1.log"

log_file_path = f"/Volumes/{config.catalog_name}/{config.schema_name}/logging/marvelous-1.log"

setup_logging(log_file=temp_log_file_path)

logger.info("Configuration loaded:")
logger.info(yaml.dump(config,default_flow_style=False))


In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
data_path = f"/Volumes/{config.catalog_name}/{config.schema_name}/files/Hotel Reservations.csv"

In [0]:
#Read in the data
with Timer() as reader_timer: #do i need to use timer with every class i use
    data_reader = DataReader(data_path)
    df = data_reader.read_csv()

logger.info(f"Data reading: {reader_timer}")

In [0]:
#Pre-process the data

with Timer() as preprocess_timer:
    data_processor = DataProcessor(df,config,spark)
    data_processor.preprocess()

logger.info(f"Data preprocessing: {preprocess_timer}")

In [0]:
#Split the data
X_train,X_test = data_processor.split_data()
logger.info(f"Training set shape: {X_train.shape}")
logger.info(f"Testing set shape: {X_test.shape}")

In [0]:
#Save to catalog
logger.info("Saving data to catalog")
data_processor.save_to_catalog(X_train,X_test)

In [0]:
with open(temp_log_file_path, "r") as log_file:
    logs = log_file.readlines()

In [0]:
dbutils.fs.put(log_file_path, "".join(logs), overwrite=True)